In [1]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt



data1 = pd.read_csv(r'2022_02_23_07_33_27.csv',encoding='gbk')
data2 = pd.read_excel(r'选择你所在家乡城市 描述统计.xlsx')
data3 = pd.read_excel(r'学校以及学院 描述统计.xlsx')
data4 = pd.read_excel(r'W020211027623974108131.xls')
data5 = pd.read_excel(r'中国一二三四五线城市名单.xlsx')

In [2]:
# 去空
for i in range(len(data4)):
    if pd.isnull(data4["Unnamed: 5"][i]):
        data4 = data4.drop(i)

In [3]:
data4

,附件1：,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
1,序号,学校名称,学校标识码,主管部门,所在地,办学层次,备注
3,1,北京大学,4111010001,教育部,北京市,本科,NaN
4,2,中国人民大学,4111010002,教育部,北京市,本科,NaN
5,3,清华大学,4111010003,教育部,北京市,本科,NaN
6,4,北京交通大学,4111010004,教育部,北京市,本科,NaN
...,...,...,...,...,...,...,...
2784,2752,阿勒泰职业技术学院,4165014680,新疆维吾尔自治区,阿勒泰地区,专科,NaN
2785,2753,塔城职业技术学院,4165014740,新疆维吾尔自治区,塔城地区,专科,NaN
2786,2754,塔里木职业技术学院,4165014741,新疆生产建设兵团,阿拉尔市,专科,NaN
2787,2755,新疆工业职业技术学院,4265051060,新疆维吾尔自治区,乌鲁木齐市,专科,NaN


In [4]:
data3

,类别,频次,百分数
0,吉林师范大学,4,0.0067
1,陕西师范大学生命科学学院,3,0.0050
2,华南师范大学,2,0.0033
3,河南护理职业学院,2,0.0033
4,四川师范大学,2,0.0033
...,...,...,...
565,中南大学交通运输工程学院,1,0.0017
566,苏州科技大学人文学院,1,0.0017
567,北京交通大学交通运输学院,1,0.0017
568,广东药科大学医药商学院,1,0.0017


In [5]:
bkyx = np.array(data4[data4["Unnamed: 5"] == "本科"]["Unnamed: 1"])
zkyx = np.array(data4[data4["Unnamed: 5"] == "专科"]["Unnamed: 1"])
bkyx

array(['北京大学', '中国人民大学', '清华大学', ..., '新疆科技学院', '新疆政法学院', '新疆天山职业技术大学'],
      dtype=object)

In [6]:
bktemp = []
zktemp = []
errotemp = []
for index, row in data3.iterrows():
    tp = row["类别"].split("大学")[0] + "大学"
    if tp in bkyx:
        bktemp.append(np.array(row))
    elif tp in zkyx:
        zktemp.append(np.array(row))
    else:
        errotemp.append(np.array(row))

In [7]:
pd.DataFrame(errotemp, columns=data3.columns).to_csv(r"./大学描述/未识别大学.csv")
pd.DataFrame(bktemp, columns=data3.columns).to_csv(r"./大学描述/本科大学.csv")
pd.DataFrame(zktemp, columns=data3.columns).to_csv(r"./大学描述/专科大学.csv")

In [8]:
tpc = data5.copy()
tem = np.array(data5["城市链接"])
temp = []
temp2 = []
for i in tem:
    if len(i.split("：")) == 2:
        temp.append(i.split("：")[1])
        temp2.append(i.split("：")[0])
    else:
        temp.append(i)
        temp2.append(i)
tpc["城市链接"] = pd.DataFrame(temp)
tpc = pd.concat([tpc, pd.DataFrame(temp2)], axis=1).drop(["Unnamed: 6", "Unnamed: 4", "全国共计337个地级以上城市（2019年5月）：" ], axis=1)
yxcity = np.array(tpc[tpc["城市分级"] == "一线城市"]["城市链接"])
xyxcity = np.array(tpc[tpc["城市分级"] == "新一线城市"]["城市链接"])
erxcity = np.array(tpc[tpc["城市分级"] == "二线城市"]["城市链接"])
sanxcity = np.array(tpc[tpc["城市分级"] == "三线城市"]["城市链接"])
sixcity = np.array(tpc[tpc["城市分级"] == "四线城市"]["城市链接"])
wuxcity = np.array(tpc[tpc["城市分级"] == "五线城市"]["城市链接"])

In [9]:
yxtemp = []
xyxtemp = []
erxtemp = []
sanxtemp = []
sixtemp = []
wuxtemp = []
errotemp = []
for index, row in data2.iterrows():
    tp1 = row["类别"].split("，")[0]
    tp2 = row["类别"].split("，")[1]
    if tp2 in yxcity:
        yxtemp.append(np.array(row))
    elif tp2 in xyxcity:
        xyxtemp.append(np.array(row))
    elif tp2 in erxcity:
        erxtemp.append(np.array(row))
    elif tp2 in sanxcity:
        sanxtemp.append(np.array(row))
    elif tp2 in sixcity:
        sixtemp.append(np.array(row))
    elif tp2 in wuxcity:
        wuxtemp.append(np.array(row))
    else:
        errotemp.append(np.array(row))

In [10]:
for i,j in zip([yxtemp, xyxtemp, erxtemp, sanxtemp, sixtemp, wuxtemp, errotemp],["一", "新一", "二","三","四","五","误"]):
    pd.DataFrame(i, columns=data2.columns).to_csv(f"./家乡城市/{j}.csv")